## The Wonderful World of ML - Session 6: Data Wrangling in R - Part 1

### Why focus on data wrangling?

In the first of these sessions, we jumped right into a few of the basic ML algorithms: linear regresssion, logistic regression, LDA, QDA, and decision trees.  In this session we'll step back and look at data wrangling.  It's an important topic because the literature suggests [[1]](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf), it consumes roughly 80% of the time spent doing an analysis.

If it takes this much time just to get the data in a usable form, it makes sense to spend a little time in building some skills in this area.




### Connect to example database

We'll use the data from the dplyr tutorial given by Hadely Wickham here: [[2]](https://www.youtube.com/watch?v=8SGif63VW6E), [[3]](https://www.youtube.com/watch?v=Ue08LVuk790).  The database has been downloaded to the project so we can connect to it locally.  Here's a bit of code to do the connection and list out the tables in the database:

In [3]:
suppressMessages(
    suppressWarnings(
        install.packages('RSQLite', repos = 'https://rweb.crmda.ku.edu/cran/')))
suppressMessages(suppressWarnings(library(DBI)))
suppressMessages(suppressWarnings(library(RSQLite)))
# connect to the sqlite file - currently in the notebooks dir of the project
# so we need to go up 2 levels to get to the data dir
con = dbConnect(SQLite(), dbname="../../data/2014_dplyr_flight_data/hflights.sqlite3")
# get a list of all tables
alltables = dbListTables(con)
alltables

[1] "airports"     "flights"      "planes"       "sqlite_stat1" "weather"

### All roads lead to Rome - Many ways to get the same result

A typical scenario which I've run into when I start some kind of analysis boils down to a series of choice that range from one of the two following extremes:

+ Write (what's often turns out to be) a complex SQL query to give you exactly what you need: both content and structure.
+ Write a simple SQL query that gets you the basic data content, but not in the right structure.  Then do a bunch of wrangling with dplyr to it into the right form.

This trade-off isn't apparent until you have do one or more joins to get the kind of dataframe you need to work on.  We'll show such an example, but for now, we'll keep it simple while we cover the basics.

### SQL Basics

Let's make a simple SQL call from R to our database and take a look at the resulting dataframe.

In [4]:
query_string <- "SELECT * FROM flights"  # get all the records in the flights table
# Fetch all query results into a data frame:
flight_data <- dbGetQuery(con, query_string)
head(flight_data)
nrow(flight_data)

# in case we need the df from the csv
#library(dplyr)
#path_flight_data <- "D:\\dev\\WonderfulML\\data\\2014_dplyr_flight_data\\flights.csv"
#flights_csv <- tbl_df(read.csv(path_flight_data, stringsAsFactors = FALSE))
#flights_csv$date <- as.Date(flights$date)

date,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
14975,14,0,1400,1500,0,-10,AA,428,DFW,N576AA,0,40,224
14976,14,1,1401,1501,1,-9,AA,428,DFW,N557AA,0,45,224
14977,13,52,1352,1502,-8,-8,AA,428,DFW,N541AA,0,48,224
14978,14,3,1403,1513,3,3,AA,428,DFW,N403AA,0,39,224
14979,14,5,1405,1507,5,-3,AA,428,DFW,N492AA,0,44,224
14980,13,59,1359,1503,-1,-7,AA,428,DFW,N262AA,0,45,224


[1] 227496

Let's fix those dates.  This is the kind of stuff you'll typically run into when you start working with a new dataset.  After reading a little of the SQLite doc's on date types [[6]](https://www.sqlite.org/datatype3.html#date_and_time_datatype), we discover that there are no explicit "date" types.  Instead, dates are expressed as either TEXT, INTEGER, or REAL types.

So how do we find out what type what types each field are in?  Again, you need some database-specific info here which you can get of the doc's.  To save a some time, you submit a query using structured like this:

In [5]:
data_types_flights <- dbGetQuery(con, "Pragma table_info (flights)")
data_types_flights

cid,name,type,notnull,dflt_value,pk
0,date,REAL,0,NA,0
1,hour,INTEGER,0,NA,0
2,minute,INTEGER,0,NA,0
3,dep,INTEGER,0,NA,0
4,arr,INTEGER,0,NA,0
5,dep_delay,INTEGER,0,NA,0
6,arr_delay,INTEGER,0,NA,0
7,carrier,TEXT,0,NA,0
8,flight,INTEGER,0,NA,0
9,dest,TEXT,0,NA,0


Looks like a real, but it can't be: 

*the number of days since noon in Greenwich on November 24, 4714 B.C. according to the proleptic Gregorian calendar*

as described in the documentation because the values are way too small.  So how do we hack these dates?

Here's where some hacking skills come in handy.  I'll bet these dates are actually INTEGER instead of REAL.

In [8]:
first_date <- as.Date("1970-01-01") + as.integer(flight_data$date[1])
first_date

[1] "2011-01-01"

If we look at the **flights.csv** file, we see that this is the date of the first record.  So we our hunch seemed on the mark.

In [14]:
query_string <- paste0("SELECT CAST(date AS INTEGER) as DATE, ",
                       "hour, minute, dep, arr, dep_delay, arr_delay, carrier, ",
                       "flight, dest, plane, cancelled, time, dist FROM flights")
flights <- dbGetQuery(con, query_string)
head(flight_data)
msg <- paste0("type before conversion: ", class(flight_data$DATE))
msg
suppressMessages(suppressWarnings(library(dplyr)))
flights <- mutate(flights, DATE = as.Date(DATE))
msg <- paste0("type AFTER conversion: ", class(flight_data$DATE))
msg
head(flight_data)

DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
14975,14,0,1400,1500,0,-10,AA,428,DFW,N576AA,0,40,224
14976,14,1,1401,1501,1,-9,AA,428,DFW,N557AA,0,45,224
14977,13,52,1352,1502,-8,-8,AA,428,DFW,N541AA,0,48,224
14978,14,3,1403,1513,3,3,AA,428,DFW,N403AA,0,39,224
14979,14,5,1405,1507,5,-3,AA,428,DFW,N492AA,0,44,224
14980,13,59,1359,1503,-1,-7,AA,428,DFW,N262AA,0,45,224


[1] "type before conversion: Date"

[1] "type AFTER conversion: Date"

DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-01-01,14,0,1400,1500,0,-10,AA,428,DFW,N576AA,0,40,224
2011-01-02,14,1,1401,1501,1,-9,AA,428,DFW,N557AA,0,45,224
2011-01-03,13,52,1352,1502,-8,-8,AA,428,DFW,N541AA,0,48,224
2011-01-04,14,3,1403,1513,3,3,AA,428,DFW,N403AA,0,39,224
2011-01-05,14,5,1405,1507,5,-3,AA,428,DFW,N492AA,0,44,224
2011-01-06,13,59,1359,1503,-1,-7,AA,428,DFW,N262AA,0,45,224


Single table dplyr verbs:

+ **filter**: keep rows matching criteria
+ **select**: pick columns by name
+ **arrange**: reorder rows
+ **mutate**: add new variables
+ **summarise**: reduce variables to values (typically used with **group_by**)

Let's look at how to use **filter**.  From page 18 in Hadley's slides [4]:

In [15]:
df <- data.frame(color = c("blue", "black", "blue", "blue", "black"), value = 1:5)
suppressMessages(suppressWarnings(library(dplyr)))
filter(df, color == "blue")

color,value
blue,1
blue,3
blue,4


Find all flights:
+ To SFO or OAK
+ In January
+ Delayed by more than an hour
+ That departed between midnight and five am.
+ Where the arrival delay was more than twice the departure delay

In [18]:
class(flights$DATE)

[1] "Date"

In [20]:
# In dplyr: step by step

p1 <- filter(flights, dest %in% c("SFO", "OAK"))
p2 <- filter(flights, DATE < "2001-02-01")  # or if ANDing together: p2 <- filter(p1, DATE < "2001-02-01")
p3 <- filter(flights, dep_delay > 60)

head(p1)
head(p2)
head(p3)

DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-01-31,8,51,851,1052,1,-27,CO,170,SFO,N35407,0,225,1635
2011-01-31,11,29,1129,1351,4,1,CO,270,SFO,N37420,0,228,1635
2011-01-31,14,32,1432,1656,7,5,CO,370,SFO,N27213,0,229,1635
2011-01-31,17,48,1748,2001,3,-4,CO,570,SFO,N75436,0,236,1635
2011-01-31,21,43,2143,2338,50,24,CO,770,SFO,N37281,0,224,1635
2011-01-31,7,29,729,1002,-1,2,CO,771,SFO,N26226,0,237,1635


DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist


DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-01-17,15,30,1530,1634,90,84,AA,428,DFW,N518AA,0,48,224
2011-01-20,15,7,1507,1622,67,72,AA,428,DFW,N425AA,0,42,224
2011-01-14,21,19,2119,2229,74,69,AA,533,DFW,N549AA,0,45,224
2011-01-09,18,35,1835,1951,125,126,AA,1121,DFW,N574AA,0,50,224
2011-01-11,17,52,1752,1855,82,70,AA,1121,DFW,N586AA,0,41,224
2011-01-10,19,34,1934,2235,99,80,AA,1294,MIA,N3BXAA,0,107,964


In [25]:
# In SQL

query1 <- paste0("SELECT * FROM flights ",
                 "WHERE dest IN ('SFO', 'OAK')") # be careful with quotes here!
query1
r1_sql <- dbGetQuery(con, query1)
head(r1_sql)

[1] "SELECT * FROM flights WHERE dest IN ('SFO', 'OAK')"

date,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
14975,17,50,1750,2002,10,2,WN,1911,OAK,N723SW,0,237,1642
14976,20,19,2019,2217,44,22,WN,1106,OAK,N407WN,0,224,1642
14976,10,43,1043,1254,28,24,WN,1892,OAK,N403WN,0,240,1642
14977,10,32,1032,1235,2,-15,WN,9,OAK,N404WN,0,230,1642
14977,21,35,2135,2337,60,42,WN,165,OAK,N232WN,0,230,1642
14978,10,34,1034,1237,4,-13,WN,9,OAK,N222WN,0,230,1642


Notice how similar this is to using SQL WHERE clauses!

Let's take a quick look at **select**.  From page 23-24 in Hadley's slides [4]:

In [26]:
select(df, color)
select(df, -color)

color
blue
black
blue
blue
black


value
1
2
3
4
5


So it's like SQL Select, but more powerful because we do things like:

In [28]:
head(select(flights, ends_with("delay")))
head(select(flights, contains("delay")))

dep_delay,arr_delay
0,-10
1,-9
-8,-8
3,3
5,-3
-1,-7


dep_delay,arr_delay
0,-10
1,-9
-8,-8
3,3
5,-3
-1,-7


How would you do this above in SQL?  Hint: It's not trivial!!!

The 3rd dplyr verb from our list is **arrange** which is sort of like an **ORDER BY** clause in SQL:

In [29]:
arrange(df, color)
arrange(df, desc(color))

color,value
black,2
black,5
blue,1
blue,3
blue,4


color,value
blue,1
blue,3
blue,4
black,2
black,5


Exercise from page 29 in Hadley's slides [4]:

+ Order the flights by departure date and time.
+ Which flights were most delayed?
+ Which flights caught up the most time during the flight?

In [32]:
r1 <- arrange(flights, DATE, hour, minute)
r2 <- arrange(flights, desc(dep_delay))
r3 <- arrange(flights, desc(arr_delay))
r4 <- arrange(flights, desc(dep_delay - arr_delay))

head(r1)
head(r2)
head(r3)
head(r4)

DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-01-01,0,1,1,621,2,24,CO,1542,SJU,N76254,0,234,2007
2011-01-01,5,49,549,846,-1,-11,DL,1248,ATL,N980DL,0,91,689
2011-01-01,5,57,557,906,-3,-9,AA,1994,MIA,N3BBAA,0,113,964
2011-01-01,5,58,558,1006,-2,-6,CO,89,EWR,N73406,0,161,1400
2011-01-01,5,59,559,758,-1,6,US,270,PHX,N313AW,0,156,1009
2011-01-01,6,0,600,657,0,-13,MQ,3265,DFW,N684MQ,0,43,247


DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-08-01,1,56,156,452,981,957,CO,1,HNL,N69063,0,461,3904
2011-12-12,6,50,650,808,970,978,AA,1740,DFW,N473AA,0,49,224
2011-11-08,7,21,721,948,931,918,MQ,3786,ORD,N502MQ,0,120,925
2011-06-21,23,34,2334,124,869,861,UA,855,SFO,N670UA,0,216,1635
2011-06-09,20,29,2029,2243,814,793,MQ,3859,ORD,N6EAMQ,0,117,925
2011-05-20,8,58,858,1027,803,822,MQ,3328,DFW,N609MQ,0,55,247


DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-12-12,6,50,650,808,970,978,AA,1740,DFW,N473AA,0,49,224
2011-08-01,1,56,156,452,981,957,CO,1,HNL,N69063,0,461,3904
2011-11-08,7,21,721,948,931,918,MQ,3786,ORD,N502MQ,0,120,925
2011-06-21,23,34,2334,124,869,861,UA,855,SFO,N670UA,0,216,1635
2011-05-20,8,58,858,1027,803,822,MQ,3328,DFW,N609MQ,0,55,247
2011-06-09,20,29,2029,2243,814,793,MQ,3859,ORD,N6EAMQ,0,117,925


DATE,hour,minute,dep,arr,dep_delay,arr_delay,carrier,flight,dest,plane,cancelled,time,dist
2011-07-03,19,14,1914,2039,-1,-70,XE,2804,MEM,N12157,0,66,468
2011-12-24,12,9,1209,1346,54,-3,CO,1669,SFO,N73406,0,201,1635
2011-12-24,21,29,2129,2337,-1,-55,CO,1552,SEA,N37437,0,234,1874
2011-12-24,21,17,2117,2258,2,-51,CO,1712,SFO,N74856,0,200,1635
2011-12-25,7,41,741,926,-4,-57,OO,4591,SLC,N814SK,0,147,1195
2011-08-26,21,7,2107,2205,-3,-55,OO,2003,BFL,N713SK,0,163,1428


The next dplyr verb is **mutate** which we've had a peek at earlier in this notebook.  This allows us to create new columns or transform existing ones.  As a simple example from page 31 in Hadley's slides [4]:

In [33]:
mutate(df, double = 2 * value)

color,value,double
blue,1,2
black,2,4
blue,3,6
blue,4,8
black,5,10


How would you create a column for the average speed?

In [38]:
# In SQL - # https://www.sqlite.org/lang_expr.html#castexpr
query2 <- "SELECT CAST(dist AS REAL)/CAST(time AS REAL) as AVG_SPEED FROM flights"  
r2_sql <- dbGetQuery(con, query2)
head(r2_sql)

# In dplyr - look ma, no cast!
r2_dplyr <- mutate(flights, AVG_SPEED = dist/time) %>% select(AVG_SPEED)
head(r2_dplyr)

AVG_SPEED
5.600000
4.977778
4.666667
5.743590
5.090909
4.977778


AVG_SPEED
5.600000
4.977778
4.666667
5.743590
5.090909
4.977778


The last dplyr verb on our list is **summerise** (if your Australian) or **summerize** (if you prefer US English).  Either on works exactly the same.  Most basic way to use this verb is without a group_by like this:

In [10]:
summarise(df, total = sum(value))

total
15


If you wanted to get the totals for each color, you'll need to do a group_by like this:

In [9]:
by_color <- group_by(df, color)
summarise(by_color, total = sum(value))

color,total
black,7
blue,8


Nice list of the summarize function on page 39 of the slides:

min(x), median(x), max(x),  
quantile(x, p)  
n(), n_distinct(), sum(x), mean(x)  
sum(x > 10), mean(x > 10)  
sd(x), var(x), IQR(x), mad(x)  

### Pipes - I didn't like these at first, but they grow on you!

First a little motivation... Can you tell what this is doing?

In [41]:
# Downside of functional interface is that it's hard to read multiple operations:
hourly_delay <- filter(
    summarise(
        group_by(filter(flights, !is.na(dep_delay)),
                 DATE, hour),
        delay = mean(dep_delay),
        n = n()),
    n > 10
)

head(hourly_delay)

DATE,hour,delay,n
2011-01-01,7,5.233333,30
2011-01-01,8,4.222222,18
2011-01-01,9,4.103448,29
2011-01-01,10,2.425532,47
2011-01-01,11,10.574468,47
2011-01-01,12,3.863636,44


### A more readable way to do the above

In [40]:
# Solution: the pipe operator from magrittr
# x %>% f(y) -> f(x, y)

hourly_delay <- flights %>%
                filter(!is.na(dep_delay)) %>%
                group_by(DATE, hour) %>%
                summarise(delay = mean(dep_delay), n = n()) %>%
                filter(n > 10)
# Hint: pronounce %>% as then

head(hourly_delay)

DATE,hour,delay,n
2011-01-01,7,5.233333,30
2011-01-01,8,4.222222,18
2011-01-01,9,4.103448,29
2011-01-01,10,2.425532,47
2011-01-01,11,10.574468,47
2011-01-01,12,3.863636,44


### References

1. Wickham, Hadley, *Tidy Data*, Journal of Statistical Software, August 2014, Volume 59, Issue 10. [https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf)
2. Hadley Wickham's "dplyr" tutorial at useR 2014 (1/2) [https://www.youtube.com/watch?v=8SGif63VW6E](https://www.youtube.com/watch?v=8SGif63VW6E)
3. Hadley Wickham's "dplyr" tutorial at useR 2014 (2/2) [https://www.youtube.com/watch?v=Ue08LVuk790](https://www.youtube.com/watch?v=Ue08LVuk790)
4. Slides for 2. and 3. are in the repo at **WonderfulML\docs\slides\2014 Wickham-dplyr-tutorial.pdf**
5. Data sets for the 2014 dplyr tutorial, [https://www.dropbox.com/sh/i8qnluwmuieicxc/AAAgt9tIKoIm7WZKIyK25lh6a](https://www.dropbox.com/sh/i8qnluwmuieicxc/AAAgt9tIKoIm7WZKIyK25lh6a)
6. RSQLite documentation: [https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf](https://cran.r-project.org/web/packages/RSQLite/RSQLite.pdf)
7. SQLite documentation regarding date types: [https://www.sqlite.org/datatype3.html#date_and_time_datatype](https://www.sqlite.org/datatype3.html#date_and_time_datatype)
8. SQLite documentation regarding table structures (pragma): [http://www.sqlite.org/pragma.html#pragma_table_info](http://www.sqlite.org/pragma.html#pragma_table_info) 
9. SQL Zoo Tutorial - http://sqlzoo.net  
10.  
11.  
12.  
